### Imports

In [1]:
import os
import sys
import yaml
import argparse
import numpy as np
import torch

from hls4ml.utils.config import config_from_pyg_model
from hls4ml.converters import convert_from_pyg_model
# module_path = os.path.abspath(os.path.join('../pyg_to_hls_hls4ml/hls4ml/utils'))
# print(module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from config import config_from_pyg_model

# module_path = os.path.abspath(os.path.join('../pyg_to_hls_hls4ml/hls4ml'))
# print(module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from converters import convert_from_pyg_model


from collections import OrderedDict
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# locals
from utils.models.interaction_network_pyg import InteractionNetwork
from model_wrappers import model_wrapper
from utils.data.dataset_pyg import GraphDataset
from utils.data.fix_graph_size import fix_graph_size

layer: Conv1d
func: <function parse_conv1d_layer at 0x7f7e36ab9680>
layer: Conv2d
func: <function parse_conv2d_layer at 0x7f7e36ab9710>
layer: Linear
func: <function parse_linear_layer at 0x7f7e36ab9830>
layer: Softmax
func: <function parse_activation_layer at 0x7f7e36ab97a0>
layer: ReLU
func: <function parse_activation_layer at 0x7f7e36ab97a0>
layer: BatchNorm2d
func: <function parse_batchnorm_layer at 0x7f7e36ab95f0>
layer: BatchNorm1d
func: <function parse_batchnorm_layer at 0x7f7e36ab95f0>
handler args: ('NodeBlock',)
handler args: ('EdgeBlock',)
handler args: ('EdgeAggregate',)
handler args: ('ResidualBlock',)
handler args: ('NodeEncoder',)
handler args: ('EdgeEncoder',)
layer: NodeBlock
func: <function parse_NodeBlock at 0x7f7e36ac6ef0>
layer: EdgeBlock
func: <function parse_EdgeBlock at 0x7f7e36acb0e0>
layer: EdgeAggregate
func: <function parse_EdgeAggregate at 0x7f7e36acb290>
layer: ResidualBlock
func: <function parse_ResidualBlock at 0x7f7e36acb440>
layer: NodeEncoder
func: <f

### PyTorch Model

In [2]:
torch_model = InteractionNetwork(aggr="add", flow="source_to_target", hidden_size=40)
# torch_model_dict = torch.load("trained_models//IN_pyg_small_add_source_to_target_40_state_dict.pt")
# torch_model.load_state_dict(torch_model_dict)

for name, submodule in torch_model.named_modules():
    if name != "":
        print(f"{name}: {submodule}")

O: ObjectModel(
  (layers): Sequential(
    (0): Linear(in_features=5, out_features=40, bias=True)
    (1): ReLU()
    (2): Linear(in_features=40, out_features=40, bias=True)
    (3): ReLU()
    (4): Linear(in_features=40, out_features=5, bias=True)
  )
)
O.layers: Sequential(
  (0): Linear(in_features=5, out_features=40, bias=True)
  (1): ReLU()
  (2): Linear(in_features=40, out_features=40, bias=True)
  (3): ReLU()
  (4): Linear(in_features=40, out_features=5, bias=True)
)
O.layers.0: Linear(in_features=5, out_features=40, bias=True)
O.layers.1: ReLU()
O.layers.2: Linear(in_features=40, out_features=40, bias=True)
O.layers.3: ReLU()
O.layers.4: Linear(in_features=40, out_features=5, bias=True)
R2: RelationalModel(
  (layers): Sequential(
    (0): Linear(in_features=15, out_features=40, bias=True)
    (1): ReLU()
    (2): Linear(in_features=40, out_features=40, bias=True)
    (3): ReLU()
    (4): Linear(in_features=40, out_features=1, bias=True)
  )
)
R2.layers: Sequential(
  (0): Lin

### HLS Model

In [3]:
# forward_dict: defines the order in which graph-blocks are called in the model's 'forward()' method
forward_dict = OrderedDict()
forward_dict["node_encoder"] = "NodeEncoder"
forward_dict["edge_encoder"] = "EdgeEncoder"
# forward_dict["R1"] = "EdgeBlock"
forward_dict["O"] = "NodeBlock"
# forward_dict["res_block"] = "ResidualBlock"
# forward_dict["R2"] = "EdgeBlock"

In [4]:
common_dim = 5
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
#         "node_dim": 3,
        "node_attr": 3,
        "node_dim": common_dim,
#         "edge_dim": 4,
        "edge_attr": 4,
    "edge_dim":common_dim
}

In [5]:
# output_dir = "test_GNN"
# config = config_from_pyg_model(torch_model,
#                                    default_precision="ap_fixed<32,12>",
#                                    default_index_precision='ap_uint<16>', 
#                                    default_reuse_factor=1)
# hls_model = convert_from_pyg_model(torch_model,
#                                        n_edge=graph_dims['n_edge'],
#                                        n_node=graph_dims['n_node'],
#                                        edge_attr=graph_dims['edge_attr'],
#                                        node_attr=graph_dims['node_attr'],
#                                        edge_dim=graph_dims['edge_dim'],
#                                        node_dim=graph_dims['node_dim'],
#                                        forward_dictionary=forward_dict, 
#                                        activate_final='sigmoid',
#                                        output_dir=output_dir,
#                                        hls_config=config)

In [6]:
output_dir = "test_GNN"
config = config_from_pyg_model(torch_model,
                                   default_precision="ap_fixed<32,8>",
                                   default_index_precision='ap_uint<16>', 
                                   default_reuse_factor=8)
print(f"config: {config}")
hls_model = convert_from_pyg_model(torch_model,
                                       n_edge=graph_dims['n_edge'],
                                       n_node=graph_dims['n_node'],
                                       edge_attr=graph_dims['edge_attr'],
                                       node_attr=graph_dims['node_attr'],
                                       edge_dim=graph_dims['edge_dim'],
                                       node_dim=graph_dims['node_dim'],
                                       forward_dictionary=forward_dict, 
                                       activate_final='sigmoid',#'sigmoid',
                                       output_dir=output_dir,
                                       hls_config=config)

config: {'Model': {'Precision': 'ap_fixed<32,8>', 'IndexPrecision': 'ap_uint<16>', 'ReuseFactor': 8, 'Strategy': 'Latency'}}
PygModelReader node_dim: 5
PygModelReader edge_dim: 5
PygModelReader node_attr: 3
PygModelReader edge_attr: 4
node encoder layer_dict['n_in']: 3
node encoder layer_dict['n_out']: 5
edge encoder b4 update_dict["last_edge_update"]: edge_attr
edge encoder after update_dict["last_edge_update"]: layer5_out
type(update_dict): <class 'dict'>
nodeblock layer_dict: {'name': 'O', 'n_node': 28, 'n_edge': 37, 'node_dim': 5, 'edge_dim': 5, 'activate_final': 'false', 'activation': 'linear', 'n_layers': 3, 'out_dim': 5, 'class_name': 'NodeBlock', 'inputs': ['layer4_out', 'layer6_out'], 'outputs': ['layer7_out']}
NodeEncoder name: node_encoder
self.state_dict.keys(): odict_keys(['O.layers.0.weight', 'O.layers.0.bias', 'O.layers.2.weight', 'O.layers.2.bias', 'O.layers.4.weight', 'O.layers.4.bias', 'R2.layers.0.weight', 'R2.layers.0.bias', 'R2.layers.2.weight', 'R2.layers.2.bias',

In [7]:
hls_model.compile()

Writing HLS project
layer: <hls4ml.model.hls_layers.Input object at 0x7f7e138cff50>
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7f7e1c123450>])
layer name: node_attr. func: None
layer: <hls4ml.model.hls_layers.Input object at 0x7f7e138d4210>
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7f7e138d42d0>])
layer name: edge_attr. func: None
layer: <hls4ml.model.hls_layers.Input object at 0x7f7e138d4250>
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7f7e138d4490>])
layer name: edge_index. func: None
layer: <hls4ml.model.hls_layers.NodeEncoder object at 0x7f7e138d4410>
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7f7e138d47d0>])
def_cpp: layer4_t layer4_out[N_LAYER_1_4*N_LAYER_2_4]
layer name: node_encoder. func: ['nnet::encoder<input_t, layer4_t, config4>(node_attr, layer4_out, w4, b4);']
layer: <hls4ml.model.hls_layers.Edg

Done
lib_name: firmware/myproject-d9A8a611.so


# Evaluation and prediction: hls_model.predict(input)

### Data

In [8]:
class data_wrapper(object):
    def __init__(self, node_attr, edge_attr, edge_index, target):
        self.x = node_attr
        self.edge_attr = edge_attr
        self.edge_index = edge_index.transpose(0,1)

        node_attr, edge_attr, edge_index = self.x.detach().cpu().numpy(), self.edge_attr.detach().cpu().numpy(), self.edge_index.transpose(0, 1).detach().cpu().numpy().astype(np.float32)
        node_attr, edge_attr, edge_index = np.ascontiguousarray(node_attr), np.ascontiguousarray(edge_attr), np.ascontiguousarray(edge_index)
        self.hls_data = [node_attr, edge_attr, edge_index]

        self.target = target
        self.np_target = np.reshape(target.detach().cpu().numpy(), newshape=(target.shape[0],))

def load_graphs(graph_indir, graph_dims, n_graphs):
    graph_files = np.array(os.listdir(graph_indir))
    graph_files = np.array([os.path.join(graph_indir, graph_file)
                            for graph_file in graph_files])
    n_graphs_total = len(graph_files)
    IDs = np.arange(n_graphs_total)
    print(f"IDS: {IDs}")
    dataset = GraphDataset(graph_files=graph_files[IDs])

    graphs = []
    for data in dataset[:n_graphs]:
        node_attr, edge_attr, edge_index, target, bad_graph = fix_graph_size(data.x, data.edge_attr, data.edge_index,
                                                                             data.y,
                                                                             n_node_max=graph_dims['n_node'],
                                                                             n_edge_max=graph_dims['n_edge'])
#         if not bad_graph:
#             graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
        graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
    print(f"graphs length: {len(graphs)}")

    print("writing test bench data for 1st graph")
    data = graphs[0]
    node_attr, edge_attr, edge_index = data.x.detach().cpu().numpy(), data.edge_attr.detach().cpu().numpy(), data.edge_index.transpose(
        0, 1).detach().cpu().numpy().astype(np.int32)
    os.makedirs('tb_data', exist_ok=True)
    input_data = np.concatenate([node_attr.reshape(1, -1), edge_attr.reshape(1, -1), edge_index.reshape(1, -1)], axis=1)
    np.savetxt('tb_data/input_data.dat', input_data, fmt='%f', delimiter=' ')

    return graphs


graph_indir = "trackml_data/processed_plus_pyg_small"
# graph_dims = {
#         "n_node": 28,
#         "n_edge": 37,
#         "node_dim": 3,
#         "edge_dim": 4
#     }
graphs = load_graphs(graph_indir, graph_dims, n_graphs=10)

IDS: [0 1 2 3 4 5 6 7 8 9]
graphs length: 10
writing test bench data for 1st graph


In [9]:
data = graphs[0]
print(f"graphs len: {len(graphs)}")
print(type(data.hls_data))
print(f"data.x shape:{data.x.shape}")
torch_pred = torch_model(data)
torch_pred = torch_pred.detach().cpu().numpy().flatten()
print(f"torch pred shape: {torch_pred.shape}")
hls_pred = hls_model.predict(data.hls_data)
print(f"hls_pred.shape: {hls_pred.shape}")
MSE = mean_squared_error(torch_pred, hls_pred)
print(f"MSE: {MSE}")

print((np.abs(torch_pred- hls_pred)))
print(torch_pred)
print(hls_pred)

graphs len: 10
<class 'list'>
data.x shape:torch.Size([28, 3])
torch pred shape: (140,)
hls_pred.shape: (140,)
MSE: 0.00037586502730846405
[5.2092671e-03 3.4529984e-02 1.7144084e-03 2.8060853e-02 9.6634626e-03
 7.6491237e-03 3.3480763e-02 1.9440353e-03 1.5950322e-02 2.0290613e-03
 5.5458546e-03 3.4673989e-02 8.2710385e-04 1.8331587e-02 7.4690580e-04
 5.3413510e-03 3.3413947e-02 2.9586554e-03 1.6622603e-02 2.0813942e-04
 5.9373677e-03 3.5144448e-02 1.7851591e-05 1.9462526e-02 2.6448965e-03
 5.5586696e-03 3.2597005e-02 1.1314154e-03 1.9545019e-02 6.9469213e-03
 5.1319599e-03 3.3728570e-02 8.8852644e-03 2.9948354e-02 2.0067453e-02
 8.8572502e-05 3.4557223e-02 4.0700436e-03 2.5431812e-02 6.3210726e-04
 3.0834675e-03 3.5500556e-02 5.7375729e-03 1.2098849e-02 1.1349499e-02
 3.3035874e-03 3.6649078e-02 4.1780770e-03 1.0579050e-02 1.4004052e-02
 2.5103986e-03 3.4463316e-02 5.6694746e-03 8.9595914e-03 1.3531923e-02
 4.0609837e-03 3.4816951e-02 1.2986064e-03 3.1275600e-02 1.0452390e-02
 5.171567

In [10]:
def get_meanNstd(data, torch_pred, hls_pred):
#     print(data.x)
    node_mean = torch.mean(data.x)
    print(f"node_mean: {node_mean}")
    node_std = torch.std(data.x)
    print(f"node_std: {node_std}")
    node_abs_max = torch.max(torch.abs(data.x))
    print(f"node_abs_max: {node_abs_max}")
    edge_mean = torch.mean(data.edge_attr)
    print(f"edge_mean: {edge_mean}")
    edge_std = torch.std(data.edge_attr)
    print(f"edge_std: {edge_std}")
    edge_abs_max = torch.max(torch.abs(data.edge_attr))
    print(f"edge_abs_max: {edge_abs_max}")
    torch_pred_mean = np.mean(torch_pred)
    print(f"torch_pred_mean: {torch_pred_mean}")
    torch_pred_std = np.std(torch_pred)
    print(f"torch_pred_std: {torch_pred_std}")
    torch_pred_abs_max = np.max(np.abs(torch_pred))
    print(f"torch_pred_abs_max: {torch_pred_abs_max}")
    hls_pred_mean = np.mean(hls_pred)
    print(f"hls_pred_mean: {hls_pred_mean}")
    hls_pred_std = np.std(hls_pred)
    print(f"hls_pred_std: {hls_pred_std}")
    hls_pred_abs_max = np.max(np.abs(hls_pred))
    print(f"hls_pred_abs_max: {hls_pred_abs_max}")
    diff_mean = np.mean(torch_pred- hls_pred)
    print(f"diff_mean: {diff_mean}")
    diff_std = np.std(torch_pred- hls_pred)
    print(f"diff_std: {diff_std}")
    diff_abs_max = np.max(np.abs(torch_pred- hls_pred))
    print(f"diff_abs_max: {diff_abs_max}")
    return None

In [11]:
MSE_l = []
for data in graphs:
#     print(f"graphs len: {len(graphs)}")
#     print(type(data.hls_data))
#     print(f"data.x shape:{data.x.shape}")
    torch_pred = torch_model(data)
    torch_pred = torch_pred.detach().cpu().numpy().flatten()
#     print(f"torch pred shape: {torch_pred.shape}")
    hls_pred = hls_model.predict(data.hls_data)
#     print(f"hls_pred.shape: {hls_pred.shape}")
    MSE = mean_squared_error(torch_pred, hls_pred)
#     get_meanNstd(data, torch_pred, hls_pred)
    print(f"MSE: {MSE}")
    MSE_l.append(MSE)
#     MAPE = mean_absolute_percentage_error(torch_pred, hls_pred)
#     print(f"MAPE: {MAPE}")

MSE_l = np.array(MSE_l)
print(f"MSE_l: {MSE_l}")
print(f"overall MSE: {np.mean(MSE_l)}")
print(MSE_l.shape)

MSE: 0.00037586502730846405
MSE: 0.00039633127744309604
MSE: 0.0003102594055235386
MSE: 0.00028307712636888027
MSE: 0.00030062245787121356
MSE: 0.0003710582386702299
MSE: 0.00038561385008506477
MSE: 0.00032143888529390097
MSE: 0.00042162497993558645
MSE: 0.0003348591853864491
MSE_l: [0.00037587 0.00039633 0.00031026 0.00028308 0.00030062 0.00037106
 0.00038561 0.00032144 0.00042162 0.00033486]
overall MSE: 0.0003500750462990254
(10,)


In [12]:
for data_instance in data.hls_data:
    print(data_instance.shape)
    
print(data.edge_index.shape)

(28, 3)
(37, 4)
(37, 2)
torch.Size([2, 37])


In [23]:
# class data_wrapper_tau3mu:
#     def __init__(self, node_attr, edge_attr, edge_index, target):
#         self.x = node_attr
#         self.edge_attr = edge_attr
#         self.edge_index = edge_index.transpose(0,1)

#         node_attr, edge_attr, edge_index = self.x.detach().cpu().numpy(), self.edge_attr.detach().cpu().numpy(), self.edge_index.transpose(0, 1).detach().cpu().numpy().astype(np.float32)
#         node_attr, edge_attr, edge_index = np.ascontiguousarray(node_attr), np.ascontiguousarray(edge_attr), np.ascontiguousarray(edge_index)
#         self.hls_data = [node_attr, edge_attr, edge_index]

#         self.target = target
#         self.np_target = np.reshape(target.detach().cpu().numpy(), newshape=(target.shape[0],))

def load_graphs_tau3mu(data_loader, graph_dims:dict, n_graphs =1000):
    """
    params:
    dataloader: pyg dataloader with custom Tau3MuDataset as its dataset
    graph_dims: 
        graph_dims.keys = ["n_node": max number of nodes allowed in a graph/batch,
            "n_edge": max number of edges allowed in a graph/batch,
            "node_dim": feature dim of node,
            "edge_dim": feature dim of edge
        ]
    
    """

    graphs = []
    
    i = 0
    for data in data_loader:
        if i >= n_graphs:
            break
        data.edge_index = data.edge_index.t()#transpose the edge_index
        n_edges = data.edge_attr.shape[0]
#         print((data.y.shape))
#         print(n_edges)
        data.y = data.y.expand(1, n_edges).flatten()
#         print(f"data.y.shape: {data.y.shape}")
        node_attr, edge_attr, edge_index, target, bad_graph = fix_graph_size(data.x, 
                                                                             data.edge_attr, 
                                                                             data.edge_index,
                                                                             data.y,
                                                                             n_node_max=graph_dims['n_node'],
                                                                             n_edge_max=graph_dims['n_edge'])
        target = torch.flatten(target)# flatten target to 1d array
        if not bad_graph:
            graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
        i +=1
        
    print(f"n_graphs: {len(graphs)}")

    print("writing test bench data for 1st graph")
    data = graphs[0]
    node_attr, edge_attr, edge_index = data.x.detach().cpu().numpy(), data.edge_attr.detach().cpu().numpy(), data.edge_index.transpose(
        0, 1).detach().cpu().numpy().astype(np.int32)
    os.makedirs('tb_data', exist_ok=True)
    input_data = np.concatenate([node_attr.reshape(1, -1), edge_attr.reshape(1, -1), edge_index.reshape(1, -1)], axis=1)
    np.savetxt('tb_data/input_data.dat', input_data, fmt='%f', delimiter=' ')

    return graphs


graph_indir = "trackml_data/processed_plus_pyg_small"
# graph_indir = "/home/swissman777/projects/Tau3MuGNNs/data"
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
        "node_dim": 3,
        "edge_dim": 4
    }


In [24]:
import pickle as pkl
from torch_geometric.loader.dataloader import DataLoader
from utils.dataset import Tau3MuDataset

# with open('./trackml_data/data_loaders.pickle', 'rb') as file:
with open('./trackml_data/data_loaders_batch_size_1.pickle', 'rb') as file:
    data_loaders= pkl.load(file) #, x_dim, edge_attr_dim 

print(data_loaders)
# for stage in data_loaders.keys():
#     for data in data_loaders[stage]:
#         print(data.x.shape)
#         print(data.edge_index.shape)
#         print(data.edge_attr.shape)

# print(graphs)

{'train': <torch_geometric.loader.dataloader.DataLoader object at 0x7f7df9e98310>, 'valid': <torch_geometric.loader.dataloader.DataLoader object at 0x7f7df9ee10d0>, 'test': <torch_geometric.loader.dataloader.DataLoader object at 0x7f7e1392a390>}


In [25]:
from sklearn.preprocessing import StandardScaler
MSE_l = []
stages = ["train", "valid", "test"]
node_scaler = StandardScaler()
edge_scaler = StandardScaler()
for stage in stages:
    graphs = load_graphs_tau3mu(data_loaders[stage], graph_dims)
#     print(type(graphs))

    for data in graphs: # fit data to scaler b4 feedforwarding
#         print(f"data type: {type(data)}")
        node_scaler.fit(data.x) 
        edge_scaler.fit(data.edge_attr) 
    
    for data in graphs:
       
        data.x = torch.from_numpy(node_scaler.transform(data.x)).float()
        data.edge_attr = torch.from_numpy(edge_scaler.transform(data.edge_attr)).float()
#         data.hls_data = [data.x, edge_attr, edge_index]
        data.hls_data[0] = node_scaler.transform(data.hls_data[0])
        data.hls_data[1] = edge_scaler.transform(data.hls_data[1])
        
        torch_pred = torch_model(data)
        torch_pred = torch_pred.detach().cpu().numpy()
        print(f"torch_pred shape: {torch_pred.shape}")
        hls_pred = hls_model.predict(data.hls_data)
        print(f"hls_pred shape: {hls_pred.shape}")
        MSE = mean_squared_error(torch_pred, hls_pred)
        print(f"MSE: {MSE}")
    #     get_meanNstd(data,torch_pred, hls_pred)
    #     print(f"MSE: {MSE}")
        MSE_l.append(MSE)
    #     MAPE = mean_absolute_percentage_error(torch_pred, hls_pred)
    #     print(f"MAPE: {MAPE}")
MSE_l = np.array(MSE_l)
print(f"MSE_l :{MSE_l}")
print(f"MSE means: {np.mean(MSE_l)}")

n_graphs: 1000
writing test bench data for 1st graph
torch_pred shape: (140,)
th n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder sta

MSE: 0.0006971106049604714
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0006305651040747762
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.00048348461859859526
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0007545742555521429
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0005774791934527457
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0005330243147909641
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0008563439478166401
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0006656830082647502
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0006280746893025935
torch_pred shape: (140,)
 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
En

MSE: 0.0008432195754721761
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0006246245466172695
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0009190311538986862
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0006222800584509969
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.000750324223190546
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0005652298568747938
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0005339801427908242
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0004737971758004278
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0004856571031268686
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0005576114635914564
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0005453647463582456
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0005441141547635198
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0010083932429552078
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0006588920368812978

torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.00047148976591415703
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0005295394803397357
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.000758271140512079
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0005205359193496406
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0007139742956496775
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0005473952041938901
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0005188121576793492
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.00043698021909222007
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.00043971603736281395
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0006464691832661629
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0006154656875878572
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0008196293492801487
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.001143410219810903
torch_pred shape: (140,)

torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.00044883586815558374
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0005119824782013893
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.001008794642984867
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0006091708783060312
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0004985917476005852
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.00044216346577741206
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0005075940862298012
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.000447689148131758
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0005298422765918076
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0004265887546353042
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.000995837734080851
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0005493228090927005
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0004695361712947488
torch_pred shape: (140,)
h

...

torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0002858076768461615
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0009299084777012467
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.00048423270345665514
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.00021017136168666184
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0009064088808372617
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0011664754711091518
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0008155762916430831
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.000454675464425236
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0008894798229448497
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.00035178608959540725
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0005944005097262561
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0006632766453549266
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0002524249139241874
torch_pred shape: (140,

torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.00046928966185078025
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0005618752911686897
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0005583771853707731
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0002745725796557963
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0004139576049055904
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0006737809162586927
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0004969715955667198
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0002727281826082617
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0008121597347781062
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0007234144723042846
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0006874897517263889
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0004592259065248072
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.00037844403414055705
torch_pred shape: (140,

torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0012270980514585972
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0011297756573185325
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.000732875894755125
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0006265313713811338
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.00021448910410981625
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0008090484188869596
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.00041043778765015304
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0006878916174173355
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0010703750886023045
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.000987633946351707
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.00010519746138015762
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0003717724175658077
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0006105718784965575
torch_pred shape: (140,)

MSE: 0.0005834472249262035
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.00034453000989742577
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0012446321779862046
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0006986293592490256
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0004144260019529611
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0002710417902562767
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0006188666447997093
torch_pred shape: (140,)
er starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encoder starting with n_in: 4 
Encoder starting with n_in: 3 
Encode

MSE: 0.0004852045967709273
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0007733752718195319
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.00035247066989541054
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0008002903778105974
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.00017588037007953972
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0007145774434320629
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0007641064585186541
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0003616661997511983
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0008914386853575706
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0008245354401879013
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0005944566801190376
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0008058426901698112
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0010491452412679791
torch_pred shape: (140,)
hls_pred shape: (140,)
MSE: 0.0006781992851756

In [26]:
MSE_l.shape

(3000,)

In [27]:
torch_model.state_dict()

OrderedDict([('O.layers.0.weight',
              tensor([[-2.3019e-01,  4.2362e-02,  3.1929e-01,  1.7338e-03,  4.0437e-01],
                      [ 4.3244e-01,  3.9680e-01,  1.4452e-01, -9.0951e-02,  2.7749e-01],
                      [-2.9764e-01,  1.0298e-01,  3.5740e-01,  2.5858e-01, -8.8100e-03],
                      [-2.0080e-01, -1.0386e-01,  4.1973e-01,  4.1643e-01,  1.9512e-01],
                      [-2.5976e-01, -3.0567e-01,  3.0110e-01, -3.0249e-02,  1.2427e-01],
                      [ 1.6785e-01, -5.8069e-02,  3.9924e-01, -1.3216e-01,  3.7029e-01],
                      [ 9.6676e-02,  2.1388e-02,  2.3036e-01,  1.2589e-01, -4.3152e-01],
                      [-2.4572e-01,  3.2313e-02,  3.2480e-01, -4.2471e-01,  4.3068e-01],
                      [-9.2165e-02, -2.5424e-01, -6.7180e-02,  4.0709e-02, -2.9878e-02],
                      [ 2.6677e-01, -3.3921e-01,  1.0485e-01,  3.6759e-01, -1.0508e-01],
                      [ 4.4351e-01, -3.9458e-01,  2.9013e-01, -5.7386e-02, 